In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
bjp_df = pd.read_csv('/content/drive/MyDrive/Data/Data/BJP_data.csv')
congress_df = pd.read_csv('/content/drive/MyDrive/Data/Data/Congress_data.csv')

In [6]:
bjp_df.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,1118007923458732032,Tue Apr 16 04:27:15 +0000 2019,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",@smitabarooah @ModiOnceMore This video show ho...,smitabarooah ModiOnceMore This video show Modi...,"Sentiment(polarity=0.6, subjectivity=0.8)",0.600000,0.80,en,0,0,wanderlust_hims,NaN,"Modi, Government, effective, works","smitabarooah, ModiOnceMore",Twitter,NaN
1,1118007893612060672,Tue Apr 16 04:27:08 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @NandiniOza: A day after his sister n fathe...,RT NandiniOza A day sister n father joined Con...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.00,en,0,1,RajeevS27545206,NaN,"Congress, BJP","NandiniOza, imjadeja, narendramodi","Delhi, India",NaN
2,1118007855020331009,Tue Apr 16 04:26:58 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",RT @dpradhanbjp: Glimpses of the massive gathe...,RT dpradhanbjp Glimpses massive gathering supp...,"Sentiment(polarity=0.0, subjectivity=1.0)",0.000000,1.00,en,0,866,Anushka97223405,NaN,"BJP, VijaySankalpYatra",dpradhanbjp,NaN,NaN
3,1118007830320242688,Tue Apr 16 04:26:52 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",RT @JhaSanjay: Dear @ArvindKejriwal Ji: Let’s ...,RT JhaSanjay Dear ArvindKejriwal Ji Let ’ clea...,"Sentiment(polarity=0.18333333333333335, subjec...",0.183333,0.85,en,0,648,YasminKidwai,NaN,BJP,"JhaSanjay, ArvindKejriwal",New Delhi,NaN
4,1118007773000716288,Tue Apr 16 04:26:39 +0000 2019,"<a href=""http://twitter.com/download/android"" ...",@kavitavkhanna @VinodKhanna @BJP4India @PMOInd...,kavitavkhanna VinodKhanna BJP4India PMOIndia n...,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000,0.00,en,0,0,vineetypmehta,NaN,BJP,"kavitavkhanna, VinodKhanna, BJP4India, PMOIndi...",India,NaN


Selecting the variables that would be used for the analysis. We select the tweet id, the text, polarity and tweet location.

In [7]:
bjp_df = bjp_df[['id','clean_text','polarity','place']]
congress_df = congress_df[['id','clean_text','polarity','place']]

**LABELING TWEETS AS POSITIVE NEGATIVE**<br>
The tweepy libary gives out sentiment polarity in the range of -1 to +1. For our topic of election prediction the neutral tweets would be of no use as they will not provide any valuable information. Thus for simplicity purpose I have labeled tweets as only positive and negative. Tweets with polarity less than 0 will be labelled negative(0) and greater than 0 will be positive(1)

In [8]:
# Convert 'polarity' column to numeric type
bjp_df['polarity'] = pd.to_numeric(bjp_df['polarity'], errors='coerce')
congress_df['polarity'] = pd.to_numeric(congress_df['polarity'], errors='coerce')

# Assign binary values based on polarity
bjp_df['polarity'] = bjp_df['polarity'].apply(lambda x: 1 if x > 0 else 0)
congress_df['polarity'] = congress_df['polarity'].apply(lambda x: 1 if x > 0 else 0)

<ipython-input-8-477746c83e89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bjp_df['polarity'] = pd.to_numeric(bjp_df['polarity'], errors='coerce')
<ipython-input-8-477746c83e89>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  congress_df['polarity'] = pd.to_numeric(congress_df['polarity'], errors='coerce')
<ipython-input-8-477746c83e89>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [9]:
bjp_df['polarity'].value_counts()

polarity
0    1655
1     613
Name: count, dtype: int64

In [10]:
congress_df['polarity'].value_counts()

polarity
0    4132
1    1676
Name: count, dtype: int64

**RESAMPLING THE DATA** <br>
Since the ratio of the negative tweets to positive tweets is not proportional. Our data set is not balanced. This will create a bias while training the model. To avoid this I have resampled the data.

In [11]:
#new data files to gather data to resample the orginal data
bjp_df_jan = pd.read_csv('/content/drive/MyDrive/Data/Data/BJP_data_jan.csv')
congress_df_jan = pd.read_csv('/content/drive/MyDrive/Data/Data/Congress_data_jan.csv')

In [12]:
bjp_df_jan = bjp_df_jan[['id','clean_text','polarity','place']]
congress_df_jan = congress_df_jan[['id','clean_text','polarity','place']]

In [13]:
bjp_df_jan['polarity'] = bjp_df_jan['polarity'].apply(lambda x: 1 if x > 0 else 0)
congress_df_jan['polarity'] = congress_df_jan['polarity'].apply(lambda x: 1 if x > 0 else 0)

<ipython-input-13-10b9057bafb3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bjp_df_jan['polarity'] = bjp_df_jan['polarity'].apply(lambda x: 1 if x > 0 else 0)


In [14]:
bjp_df_jan['polarity'].value_counts()

polarity
0    1303
1     853
Name: count, dtype: int64

In [15]:
congress_df_jan['polarity'].value_counts()

polarity
0    857
1    352
Name: count, dtype: int64

In order to balance the data we need to add only positive tweets into our original data. From the new data, I will only pick the positive tweets and then add them to our main data.

In [16]:
bjp_df_jan =bjp_df_jan.loc[bjp_df_jan['polarity'] == 1]
congress_df_jan =congress_df_jan.loc[congress_df_jan['polarity'] == 1]

In [17]:
# Appending positive tweets to the original dataframes
bjp = pd.concat([bjp_df, bjp_df_jan], ignore_index=True)
congress = pd.concat([congress_df, congress_df_jan], ignore_index=True)

In [18]:
bjp['polarity'].value_counts()

polarity
0    1655
1    1466
Name: count, dtype: int64

In [19]:
congress['polarity'].value_counts()

polarity
0    4132
1    2028
Name: count, dtype: int64

The above data is now sampled and the proportion of both negative and positive tweets is balanced

In [20]:
bjp.to_csv('bjp_final',index= False)
congress.to_csv('congress_final', index = False)

Now, I will combine the congress and bjp data frames to create a single dataframe which will be useful in our future analysis.

**CREATING FINAL DATASET**

In [21]:
frames = [bjp, congress]
election_data = pd.concat(frames)

In [22]:
#saving the Final data into a csv
election_data.to_csv('election',index=False)